In [4]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="protest-impact")
location = geolocator.geocode("Baden-Württemberg")
location.address, (location.latitude, location.longitude), location.raw

('Baden-Württemberg, Deutschland',
 (48.53775, 9.041169),
 {'place_id': 298120027,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 62611,
  'boundingbox': ['47.5324787', '49.7912941', '7.5117461', '10.4955731'],
  'lat': '48.53775',
  'lon': '9.041169',
  'display_name': 'Baden-Württemberg, Deutschland',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.8889406218540863,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'})

In [8]:
from protest_impact.data import german_regions

coordinates = {
    a["name"]: geolocator.geocode(a["capital"] + ", Germany") for a in german_regions
}
coordinates

{'Baden-Württemberg': Location(Stuttgart, Baden-Württemberg, Deutschland, (48.7784485, 9.1800132, 0.0)),
 'Bayern': Location(München, Bayern, Deutschland, (48.1371079, 11.5753822, 0.0)),
 'Berlin': Location(Berlin, Deutschland, (52.5170365, 13.3888599, 0.0)),
 'Brandenburg': Location(Potsdam, Brandenburg, Deutschland, (52.4009309, 13.0591397, 0.0)),
 'Bremen': Location(Bremen, Deutschland, (53.0758196, 8.8071646, 0.0)),
 'Hamburg': Location(Hamburg, Deutschland, (53.550341, 10.000654, 0.0)),
 'Hessen': Location(Wiesbaden, Hessen, Deutschland, (50.0820384, 8.2416556, 0.0)),
 'Mecklenburg-Vorpommern': Location(Schwerin, Mecklenburg-Vorpommern, Deutschland, (53.6288297, 11.4148038, 0.0)),
 'Niedersachsen': Location(Hannover, Region Hannover, Niedersachsen, Deutschland, (52.3744779, 9.7385532, 0.0)),
 'Nordrhein-Westfalen': Location(Düsseldorf, Nordrhein-Westfalen, Deutschland, (51.2254018, 6.7763137, 0.0)),
 'Rheinland-Pfalz': Location(Mainz, Rheinland-Pfalz, Deutschland, (50.0012314, 8.2

In [9]:
coordinates = {k: (v.latitude, v.longitude) for k, v in coordinates.items()}
coordinates

{'Baden-Württemberg': (48.7784485, 9.1800132),
 'Bayern': (48.1371079, 11.5753822),
 'Berlin': (52.5170365, 13.3888599),
 'Brandenburg': (52.4009309, 13.0591397),
 'Bremen': (53.0758196, 8.8071646),
 'Hamburg': (53.550341, 10.000654),
 'Hessen': (50.0820384, 8.2416556),
 'Mecklenburg-Vorpommern': (53.6288297, 11.4148038),
 'Niedersachsen': (52.3744779, 9.7385532),
 'Nordrhein-Westfalen': (51.2254018, 6.7763137),
 'Rheinland-Pfalz': (50.0012314, 8.2762513),
 'Saarland': (49.234362, 6.996379),
 'Sachsen': (51.0493286, 13.7381437),
 'Sachsen-Anhalt': (52.1315889, 11.6399609),
 'Schleswig-Holstein': (54.3227085, 10.135555),
 'Thüringen': (50.9777974, 11.0287364)}

In [14]:
from geopy.distance import geodesic

inverse_distances = {
    a["name"]: {
        b["name"]: 1 / geodesic(coordinates[a["name"]], coordinates[b["name"]]).km
        for b in german_regions
        if a["name"] != b["name"]
    }
    for a in german_regions
}
inverse_distances["Berlin"]

{'Baden-Württemberg': 0.0019559762930516297,
 'Bayern': 0.001984199049647133,
 'Brandenburg': 0.03865571419402733,
 'Bremen': 0.003172767587826417,
 'Hamburg': 0.003926274354943075,
 'Hessen': 0.002224422534152766,
 'Mecklenburg-Vorpommern': 0.0055207371081691286,
 'Niedersachsen': 0.004021266717007122,
 'Nordrhein-Westfalen': 0.002094803669581934,
 'Rheinland-Pfalz': 0.0022057139214784285,
 'Saarland': 0.0017267897738956527,
 'Sachsen': 0.006058012194805484,
 'Sachsen-Anhalt': 0.007891658904078336,
 'Schleswig-Holstein': 0.0033876943704335945,
 'Thüringen': 0.0042300419330770864}

In [15]:
weights = {
    a["name"]: {
        b["name"]: inverse_distances[a["name"]][b["name"]]
        / sum(inverse_distances[a["name"]].values())
        for b in german_regions
        if a["name"] != b["name"]
    }
    for a in german_regions
}
weights["Berlin"]

{'Baden-Württemberg': 0.021963424119334034,
 'Bayern': 0.02228033408144645,
 'Brandenburg': 0.4340603965882384,
 'Bremen': 0.03562662820150479,
 'Hamburg': 0.044087665669986315,
 'Hessen': 0.024977774890092393,
 'Mecklenburg-Vorpommern': 0.06199169743969062,
 'Niedersachsen': 0.045154323555115866,
 'Nordrhein-Westfalen': 0.023522300145051318,
 'Rheinland-Pfalz': 0.024767698113441,
 'Saarland': 0.01938991607604216,
 'Sachsen': 0.0680246952006889,
 'Sachsen-Anhalt': 0.08861449503816464,
 'Schleswig-Holstein': 0.03804001536666342,
 'Thüringen': 0.047498635514539596}

In [17]:
import json

with open("../../data/regions/region-weights.json", "w") as f:
    json.dump(weights, f)